This notebook is part of the Practical Cheminformatics tutorials series.  The complete series can be found at [this link](https://github.com/PatWalters/practical_cheminformatics_tutorials).

### Streamlining Cheminformatics Workflows with [datamol](https://datamol.io) 1. Data manipulation, descriptors and clustering 

The [datamol](datamol.io) package from the M2D2 group provides a set of functions that simplify some common Cheminformatics tasks.  The best part of this package is that it builds on the RDKit. All the RDKit functionality you know and love is there, it's just easier to access.  In this notebook I'll highlight some basic data manipulation capabilities available in datamol.
- Transforming units
- Adding molecules and descriptors to Pandas dataframes
- Filtering dataframes based on descriptor values
- Clustering

Install the necessary Python libraries

In [ ]:
!pip install pandas rdkit datamol tqdm mols2grid

Import the necessary Python libraries

In [1]:
import pandas as pd
import datamol as dm
from tqdm.auto import tqdm
from rdkit.Chem import rdDepictor
import mols2grid

### A few preliminary settings
A couple settings to get us started.  Let's make sure our chemical structures will look good and that floating point values aren't shown with a ridiculous number of decimal places.

In [2]:
rdDepictor.SetPreferCoordGen(True)
pd.options.display.float_format = '{:,.2f}'.format

### Reading data
Read an input dataset.  There are a couple of things to notice here.  
- The input data doesn't have column headers, we use the "names" argument to [**pd.read_csv**](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) to specify column names.
- We are using a space as a delimiter, this is specified in the **sep** argument.

In [3]:
url = "https://raw.githubusercontent.com/PatWalters/yamc/main/data/HERG.smi"
df = pd.read_csv(url,sep=" ",names=["SMILES","Name","pIC50"])

In [4]:
df.head()

,SMILES,Name,pIC50
0,c1cc(ccc1n2cc(c3c2ccc(c3)Cl)C4CCN(CC4)CCN5CCNC...,CHEMBL12713,8.03
1,c1ccc2c(c1)[nH]c(=O)n2C3=CCN(CC3)CCCC(=O)c4ccc...,CHEMBL1108,7.33
2,c1ccc2c(c1)c(c[nH]2)C(=O)OC3CC4CC5CC(C3)N4CC5=O,CHEMBL2368925,5.07
3,CC(C)C(CCCN(C)CCc1ccc(c(c1)OC)OC)(C#N)c2ccc(c(...,CHEMBL6966,6.83
4,CCOC(=O)N1CCC(=C2c3ccc(cc3CCc4c2nccc4)Cl)CC1,CHEMBL998,6.43


### Transforming units
This data has the pIC50, or the log10 of the IC50 in $\mu$M.  Whenever I look at pIC50 values I end up mentally converting them to IC50 values.  I've become pretty good at this, but I'd prefer to have the IC50 values in the table. Fortunately, datamol has a set of convenience methods in the [**dm.molar**](https://docs.datamol.io/0.8.5/api/datamol.molar.html) package that do this conversion.  This isn't something that's hard to do, but it's nice to have a simple function available to handle the conversion. For more on **dm.molar** consult the docs.
```python
help(dm.molar)
```

In [5]:
df['IC50'] = dm.molar.log_to_molar(df.pIC50,'uM')
df.head()

,SMILES,Name,pIC50,IC50
0,c1cc(ccc1n2cc(c3c2ccc(c3)Cl)C4CCN(CC4)CCN5CCNC...,CHEMBL12713,8.03,0.01
1,c1ccc2c(c1)[nH]c(=O)n2C3=CCN(CC3)CCCC(=O)c4ccc...,CHEMBL1108,7.33,0.05
2,c1ccc2c(c1)c(c[nH]2)C(=O)OC3CC4CC5CC(C3)N4CC5=O,CHEMBL2368925,5.07,8.48
3,CC(C)C(CCCN(C)CCc1ccc(c(c1)OC)OC)(C#N)c2ccc(c(...,CHEMBL6966,6.83,0.15
4,CCOC(=O)N1CCC(=C2c3ccc(cc3CCc4c2nccc4)Cl)CC1,CHEMBL998,6.43,0.37


### Adding a molecule to a dataframe
The datamol function [**from_df**](https://docs.datamol.io/stable/api/datamol.convert.html) provides a convenient means of adding an RDKit molecule column to a dataframe.

In [6]:
df['mol'] = dm.from_df(df, smiles_column="SMILES")
df.head()

,SMILES,Name,pIC50,IC50,mol
0,c1cc(ccc1n2cc(c3c2ccc(c3)Cl)C4CCN(CC4)CCN5CCNC...,CHEMBL12713,8.03,0.01,<rdkit.Chem.rdchem.Mol object at 0x29d602a50>
1,c1ccc2c(c1)[nH]c(=O)n2C3=CCN(CC3)CCCC(=O)c4ccc...,CHEMBL1108,7.33,0.05,<rdkit.Chem.rdchem.Mol object at 0x29d602740>
2,c1ccc2c(c1)c(c[nH]2)C(=O)OC3CC4CC5CC(C3)N4CC5=O,CHEMBL2368925,5.07,8.48,<rdkit.Chem.rdchem.Mol object at 0x29d602b30>
3,CC(C)C(CCCN(C)CCc1ccc(c(c1)OC)OC)(C#N)c2ccc(c(...,CHEMBL6966,6.83,0.15,<rdkit.Chem.rdchem.Mol object at 0x29d602820>
4,CCOC(=O)N1CCC(=C2c3ccc(cc3CCc4c2nccc4)Cl)CC1,CHEMBL998,6.43,0.37,<rdkit.Chem.rdchem.Mol object at 0x29d602970>


### Calculating simple 1D descriptors
The datamol package has a function [**batch_compute_many_descriptors**](https://docs.datamol.io/stable/api/datamol.descriptors.html) that calculates many of the molecular descriptors supported by the RDKit.  When called with the default arguments, this function calculates 22 different descriptors. While this is useful, I usually only want to add a few descriptors to a dataframe.  We can do this by passing a dictionary to the **properties_fn** argument of **batch_compute_many_descriptors**.  The key for this dictionary is the name of the resulting dataframe column and the value is the function that will be called to calculate the descriptor. This function should take an RDKit molecule as an argument. These functions don't necessarily have to be datamol or RDKit functions.  You can also add your own functions. For instance let's define a function to get the size of the largest ring in a molecule.

In [7]:
def max_ring_size(mol):
    """Get the size of the largest ring in a molecule

    :param mol: input_molecule
    :return: size of the largest ring or 0 for an acyclic molecule
    """
    ri = mol.GetRingInfo()
    atom_rings = ri.AtomRings()
    if len(atom_rings) == 0:
        return 0
    else:
        return max([len(x) for x in ri.AtomRings()])

Now we can define the dictionary with the properties we want to calculate.

In [8]:
my_prop_dict = {
    "mw" : dm.descriptors.mw,
    "logp" : dm.descriptors.clogp,
    "hbd" : dm.descriptors.n_lipinski_hbd,
    "hba" : dm.descriptors.n_lipinski_hba,
    "max_ring_size" : max_ring_size
}

The datamol function **batch_compute_many_descriptors** returns a new dataframe with the computed descriptors. Note a couple of parameter settings.
- add_properties=False - if this is set to True, all the supported descriptors are calculated
- progress=True - show a progress bar

In [9]:
prop_df = dm.descriptors.batch_compute_many_descriptors(df.mol,properties_fn=my_prop_dict,add_properties=False,
                                             progress=True)

  0%|          | 0/4042 [00:00<?, ?it/s]

Now we have a dataframe with the properties, but it doesn't have any of the information from our original dataframe.  We need to merge the two dataframes. 

In [10]:
prop_df

,mw,logp,hbd,hba,max_ring_size
0,440.18,4.63,1,5,6
1,379.17,3.68,1,5,6
2,324.15,2.52,1,5,6
3,454.28,5.09,0,6,6
4,382.14,4.89,0,4,7
...,...,...,...,...,...
4037,493.21,3.38,2,10,6
4038,480.18,3.16,2,10,6
4039,494.19,3.81,1,10,6
4040,494.19,3.55,2,10,6


Merge the descriptors into the main dataframe.

In [11]:
df = pd.concat([df,prop_df],axis=1)

### Filtering a dataframe based on descriptor values 
Let's filter the dataframe to only include molecules meeting the Lipinski Rule of 5 (Ro5) criteria.  I see a lot of people doing this using the Pandas bracket notation to do something like this.  
```python
df = df[df['mw'] <= 500]
df = df[df['logp'] <= 5]
df = df[df['hbd'] <= 5]
df = df[df['hba'] <= 10]
```
To me, this feels convoluted. When I try to do this, I usuually forget the syntax or screw something up.  I prefer using the pandas [**query**](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.query.html) function.

In [12]:
df_ro5_ok = df.query("mw <= 500 and logp <= 5 and hbd <= 5 and hba <= 10")
len(df_ro5_ok)

3476

We could also create a column indicating whether a molecule passed the Ro5

In [13]:
df['ro5_ok'] = (df.mw <= 500) & (df.logp <=5) & (df.hbd <= 5) & (df.hba <= 10)
df.ro5_ok.sum()

3476

What if we want to count how many of the Ro5 criteria a molecule meets?  We could simply write a function to convert a boolean column to integers and calculate the sum of these integer columns. 

In [14]:
def bool_to_int(bool_in):
    return bool_in.values.astype(int)

Now we can use the function above to count how many of the Ro5 rules a molecule passes.  A molecule that passes all the rules will have **ro5_count** set to 4.  As a check we can make sure the number of molecules passing all the rules is the same as what we calculated above.

In [15]:
df['ro5_count'] = bool_to_int(df.mw <= 500) + bool_to_int(df.logp <=5) + bool_to_int(df.hbd <= 5) + bool_to_int(df.hba <= 10)
len(df.query("ro5_count == 4"))

3476

### Clustering
The datamol package provides a couple of useful functions for clustering.  The [**cluster_mols**](https://docs.datamol.io/stable/api/datamol.cluster.html) function provides a simple means of clustering based on the [Butina](https://pubs.acs.org/doi/pdf/10.1021/ci9803381) algorithm using the RDKit's Morgan fingerprints.  You can perform Butina clustering with the RDKit, but it takes a few more steps.

When I look at the results of clustering, I like to see similar molecules aligned in a similar fashion.  Datamol has another convenience function [**auto_align_many**](https://docs.datamol.io/stable/api/datamol.align.html), that aligns molecules with the same scaffold.  Note that you can do clustering and alignment in one step with **auto_align_many**.  However, I prefer to do this as a two-step process that gives me a bit more control over the clustering and the alignment. Both **cluster_mols** and **auto_align_many** have several options.  Check out the docs for more information.

In [16]:
# cluster the molecules
cluster_list = dm.cluster_mols(df.mol)
# create a list to hold the cluster ids, which we will add to the dataframe
cluster_idx = [-1] * len(df)
for i,cluster in enumerate(tqdm(cluster_list[0])):
    # align the structures for each cluster using Bemis Murcko frameworks
    dm.align.auto_align_many([df.mol.values[x] for x in cluster],copy=False,partition_method='scaffold')
    # add the cluster id to cluster_idx
    for c in cluster:
        cluster_idx[c] = i
# add a column with cluster ids to the dataframe
df['cluster'] = cluster_idx

  0%|          | 0/3503 [00:00<?, ?it/s]

To get an overview of the data we'll create a dataframe with one example from each cluster.

In [17]:
cluster_sample_df = df.sort_values("cluster").drop_duplicates("cluster").copy()
cluster_sample_df.head()

,SMILES,Name,pIC50,IC50,mol,mw,logp,hbd,hba,max_ring_size,ro5_ok,ro5_count,cluster
3888,c1ccc(cc1)c2ccc(cc2)c3c[nH]c(n3)C4Cc5c6ccccc6[...,CHEMBL3775414,7.77,0.02,<rdkit.Chem.rdchem.Mol object at 0x29da1db30>,474.24,6.58,3,5,6,False,3,0
1119,COc1ccc(cc1)CCN2C(CN(C2=O)NS(=O)(=O)C)c3ccc(cc...,CHEMBL571041,5.04,9.12,<rdkit.Chem.rdchem.Mol object at 0x29d6bac80>,457.13,3.20,1,7,6,True,4,1
2356,CN1CCN(CC1)CCCNc2nnc(o2)c3ccc(cc3)NC(=O)c4ccccc4F,CHEMBL2022501,4.20,63.10,<rdkit.Chem.rdchem.Mol object at 0x29d6e7270>,438.22,3.18,2,8,6,True,4,2
1230,Cc1c(ocn1)c2nnc(n2C)SCCCN3CC4CC4(C3)c5ccc(cc5)...,CHEMBL1081966,6.60,0.25,<rdkit.Chem.rdchem.Mol object at 0x29d6bedd0>,479.16,4.43,0,7,6,True,4,3
1222,Cc1c(ocn1)c2nnc(n2C)SCCCN3CC4CC4(C3)c5ccc(cc5)Cl,CHEMBL1080490,6.39,0.41,<rdkit.Chem.rdchem.Mol object at 0x29d6bea50>,429.14,4.19,0,6,6,True,4,4


I'd also like to see the size of each cluster.  We can use the Pandas [value_counts](https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html) function to create a new dataframe with the cluster number and the number of molecules in each cluster.  We can then use the pandas [merge](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html) function to combine **cluster_sample_df** and **cluster_count_df**.

In [18]:
cluster_count_df = df.cluster.value_counts().to_frame().reset_index()
cluster_count_df.columns = ["cluster","count"]
cluster_sample_df = cluster_sample_df.merge(cluster_count_df,on="cluster")
cluster_sample_df.head()

,SMILES,Name,pIC50,IC50,mol,mw,logp,hbd,hba,max_ring_size,ro5_ok,ro5_count,cluster,count
0,c1ccc(cc1)c2ccc(cc2)c3c[nH]c(n3)C4Cc5c6ccccc6[...,CHEMBL3775414,7.77,0.02,<rdkit.Chem.rdchem.Mol object at 0x29da1db30>,474.24,6.58,3,5,6,False,3,0,10
1,COc1ccc(cc1)CCN2C(CN(C2=O)NS(=O)(=O)C)c3ccc(cc...,CHEMBL571041,5.04,9.12,<rdkit.Chem.rdchem.Mol object at 0x29d6bac80>,457.13,3.20,1,7,6,True,4,1,10
2,CN1CCN(CC1)CCCNc2nnc(o2)c3ccc(cc3)NC(=O)c4ccccc4F,CHEMBL2022501,4.20,63.10,<rdkit.Chem.rdchem.Mol object at 0x29d6e7270>,438.22,3.18,2,8,6,True,4,2,9
3,Cc1c(ocn1)c2nnc(n2C)SCCCN3CC4CC4(C3)c5ccc(cc5)...,CHEMBL1081966,6.60,0.25,<rdkit.Chem.rdchem.Mol object at 0x29d6bedd0>,479.16,4.43,0,7,6,True,4,3,8
4,Cc1c(ocn1)c2nnc(n2C)SCCCN3CC4CC4(C3)c5ccc(cc5)Cl,CHEMBL1080490,6.39,0.41,<rdkit.Chem.rdchem.Mol object at 0x29d6bea50>,429.14,4.19,0,6,6,True,4,4,4


We can use [mols2grid](https://github.com/cbouy/mols2grid) to display and scroll through the cluster samples

In [19]:
# This should work but the grid doesn't display on Colab
# mols2grid.display(cluster_sample_df,mol_col="mol",subset=["img","cluster","count"],use_coords=True, prerender=True)
# As workaround we have to do this on Colab.  If you're running on your own machine, substitute the line above
mols2grid.display(cluster_sample_df,mol_col="mol",subset=["img","cluster","count"])

MolGridWidget()

If we see an interesting cluster, we can use the function **show_cluster** to examine the molecules in that cluster.

In [20]:
def show_cluster(x):
    return mols2grid.display(df.query(f"cluster == {x}"),mol_col="mol",subset=["img","Name","pIC50"],use_coords=True, prerender=True, transform = {"pIC50" : lambda val: f"{val:.2f}"})

In [21]:
show_cluster(1)

MolGridWidget()